# Notebook for visualizing how backward pass is performed step by step through different layers with comparison with torch autograd

In [28]:
from abc import ABC, abstractmethod
import copy
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from typing import Union
from IPython.display import HTML, clear_output

In [2]:
BOS, EOS = ' ', '\n'

data = pd.read_json("./arxivData.json")
lines = data.apply(lambda row: (row['title'] + ' ; ' + row['summary'])[:128], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()

tokens = list(set(''.join(lines)))

num_tokens = len(tokens)
print('num_tokens = ', num_tokens)

token_to_id = {token: idx for idx, token in enumerate(tokens)}

def to_matrix(data, token_to_id, max_len=None, dtype='int32', batch_first=True):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, data))
    data_ix = np.zeros([len(data), max_len], dtype) + token_to_id[' ']

    for i in range(len(data)):
        line_ix = [token_to_id[c] for c in data[i]]
        data_ix[i, :len(line_ix)] = line_ix
        
    if not batch_first: # convert [batch, time] into [time, batch]
        data_ix = np.transpose(data_ix)

    return data_ix

num_tokens =  136


In [3]:
class BaseLayer(ABC):
    @abstractmethod
    def __init__(self) -> None:
        pass

    def __call__(self, x: np.array, grad: bool = True) -> np.array:
        return self.forward(x, grad)

    @abstractmethod
    def forward(self, x: np.array, grad: bool = True) -> np.array:
        pass

    @abstractmethod
    def backward(self, output_error: np.array) -> np.array:
        pass

# emb check

In [6]:
class Embedding(BaseLayer):
    def __init__(self, n_input, emb_dim, pad_idx=None):
        self.n_input = n_input
        self.emb_dim = emb_dim
        self.pad_idx = pad_idx
        
        self.weights = np.random.normal(scale=np.sqrt(2/(n_input+emb_dim)), size=(n_input, emb_dim))

    def set_optimizer(self, optimizer):
        self.weights_optimizer = copy.copy(optimizer)

        self.weights_optimizer.set_weight(self.weights)

    def forward(self, x, grad=True):
        self.input = x
        return self.weights[x]

    def backward(self, output_error):
        weights_grad = np.zeros_like(self.weights)
        input_shape_len = len(self.input.shape)

        if input_shape_len == 2:
            for batch_n, s in enumerate(self.input):
                for i, emb_i in enumerate(s):
                    weights_grad[emb_i] += output_error[batch_n][i]

        elif input_shape_len == 1:
            for i, emb_i in enumerate(self.input):
                weights_grad[emb_i] += output_error[i]

        if self.pad_idx is not None:
            weights_grad[self.pad_idx] = 0

        # self.weights = self.weights_optimizer.step(weights_grad)
        return weights_grad

In [7]:
# проверка на то, что градиент эмбеддингов считается правильно

emb = Embedding(len(token_to_id), 16)
print("Embeddings shape:", emb.weights.shape)

torch_emb = torch.nn.Embedding(len(token_to_id), 16)
torch_emb.weight.data = torch.as_tensor(emb.weights)

torch_out = torch_emb(torch.as_tensor(sample))

print("Forward совпадает:", np.allclose(torch_out.detach().numpy(), emb(sample)))

check_error = np.random.normal(0, 100, torch_out.shape)
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)

print("Градиенты совпадают", np.allclose(torch_emb.weight.grad.detach().numpy(), emb.backward(check_error)))

Embeddings shape: (136, 16)
Forward совпадает: True
Градиенты совпадают True


# conv1d check

In [8]:
class Conv1dVanilla(BaseLayer):
    """
    Сверточный слой, со страйдом 1 и без паддингов, для 1 предложения, не батча
    """
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        scale = np.sqrt(1/(in_channels*kernel_size))
        self.kernel = np.random.uniform(-scale, scale, size=(out_channels, in_channels, kernel_size))
        self.bias = np.random.uniform(-scale, scale, size=(out_channels))

    def set_optimizer(self, optimizer):
        self.kernel_optimizer = copy.copy(optimizer)
        self.bias_optimizer = copy.copy(optimizer)

        self.kernel_optimizer.set_weight(self.kernel)
        self.bias_optimizer.set_weight(self.bias)

    def forward(self, x, grad=True):
        self.input = x

        self.output_len = x.shape[0] - self.kernel_size + 1
        output = np.zeros(shape=(self.output_len, self.out_channels))

        for kernel_i, ker in enumerate(self.kernel):
            for i in range(self.output_len):
                output[i:self.kernel_size+i, kernel_i] = self.bias[kernel_i] + np.sum(x[i:self.kernel_size+i, :] * ker.T)

        return output

    def backward(self, output_error):
        dy_dkernel = np.zeros(shape=self.kernel.shape)
        dy_dbias = np.zeros(shape=self.bias.shape)
        dy_dx = np.zeros(shape=self.input.shape)

        for kernel_i, ker in enumerate(self.kernel):
            helper_k = np.zeros(shape=ker.T.shape)

            for i in range(self.output_len):
                helper_k += self.input[i:self.kernel_size+i, :] * output_error[i, kernel_i]
                dy_dx[i:self.kernel_size+i, :] += ker.T * output_error[i, kernel_i]

            dy_dkernel[kernel_i] = helper_k.T
            dy_dbias[kernel_i] = np.sum(output_error[:, kernel_i])

        # self.kernel = self.kernel_optimizer.step(dy_dkernel)
        # self.bias = self.bias_optimizer.step(dy_dbias)

        # return dy_dx
        return dy_dkernel, dy_dbias, dy_dx

In [9]:
sample = to_matrix(np.random.choice(lines, size=5), token_to_id, max_len=130)

emb = Embedding(len(token_to_id), 16)
encoded_data = emb(sample[0])

conv = Conv1dVanilla(16, 4, 5)
torch_conv = torch.nn.Conv1d(16, 4, 5)
torch_conv.weight.data = torch.as_tensor(conv.kernel)
torch_conv.bias.data = torch.as_tensor(conv.bias)

torch_input = torch.tensor(encoded_data[np.newaxis, :], dtype=torch.float64, requires_grad=True)
torch_out = torch_conv(torch_input.permute(0, 2, 1))  # permute потому что torch на вход принимает формат [BATCH_SIZE, EMB_DIM, SENTENCE_LEN]

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.permute(0, 2, 1).detach().numpy(), conv(encoded_data)))

print("Shape выхода свертки:", conv(encoded_data).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(conv(encoded_data).shape))
check_error_torch = torch.tensor(np.transpose(check_error[np.newaxis, :], (0, 2, 1)))

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = conv.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_conv.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_conv.weight.grad.detach().numpy(), kernel_grad))

Vanilla Forward такой же как и torch forward: True
Shape выхода свертки: (126, 4)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [10]:
class Conv1d(BaseLayer):
    """
    Сверточный слой, со страйдом 1 и без паддингов, для батча
    """
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        scale = np.sqrt(1/(in_channels*kernel_size))
        self.kernel = np.random.uniform(-scale, scale, size=(out_channels, in_channels, kernel_size))
        self.bias = np.random.uniform(-scale, scale, size=(out_channels))

    def set_optimizer(self, optimizer):
        self.kernel_optimizer = copy.copy(optimizer)
        self.bias_optimizer = copy.copy(optimizer)

        self.kernel_optimizer.set_weight(self.kernel)
        self.bias_optimizer.set_weight(self.bias)

    def forward(self, x, grad=True):
        """
        Работает с битчами вида [BATCH_SIZE, SENTENCE_LEN, EMB_DIM]
        """
        self.input = x
        self.batch_size = x.shape[0]
        self.input_len = x.shape[1]
        self.output_len = self.input_len - self.kernel_size + 1

        result = []

        for sentence in x:
            result.append(self._forward_for_one(sentence))

        return np.array(result)

    def _forward_for_one(self, x):
        """
        Просто свертка для 1 предложения
        """
        output = np.zeros(shape=(self.output_len, self.out_channels))

        # для каждого выходного канала и ядра, отвечающего за этот канал
        for kernel_i, ker in enumerate(self.kernel):
            # по выходной длине
            for i in range(self.output_len):
                # умножаем срез по размеру ядра на ядро и суммируем
                output[i:self.kernel_size+i, kernel_i] = self.bias[kernel_i] + np.sum(x[i:self.kernel_size+i, :] * ker.T)

        return output

    def backward(self, output_error):
        """
        Градиенты по всем батчу
        """
        dy_dkernels = []
        dy_dbiass = []
        dy_dxs = []

        for i in range(self.batch_size):
            dy_dkernel, dy_dbias, dy_dx = self._calc_grad_for_one(output_error[i], self.input[i])
            dy_dkernels.append(dy_dkernel)
            dy_dbiass.append(dy_dbias)
            dy_dxs.append(dy_dx)

        dy_dkernels = np.sum(np.array(dy_dkernels), axis=0)  # суммируем градиенты по батчу
        dy_dbiass = np.sum(np.array(dy_dbiass), axis=0)
        dy_dxs = np.array(dy_dxs)

        # self.kernel = self.kernel_optimizer.step(dy_dkernels)  # делаем шаг спуска по сумме градиентов
        # self.bias = self.bias_optimizer.step(dy_dbiass)

        # return dy_dxs
        return dy_dkernels, dy_dbiass, dy_dxs

    def _calc_grad_for_one(self, output_error, x):
        dy_dkernel = np.zeros(shape=self.kernel.shape)
        dy_dbias = np.zeros(shape=self.bias.shape)
        dy_dx = np.zeros(shape=x.shape)

        for kernel_i, ker in enumerate(self.kernel):
            helper_k = np.zeros(shape=ker.T.shape)

            for i in range(self.output_len):
                helper_k += x[i:self.kernel_size+i, :] * output_error[i, kernel_i]
                dy_dx[i:self.kernel_size+i, :] += ker.T * output_error[i, kernel_i]

            dy_dkernel[kernel_i] = helper_k.T
            dy_dbias[kernel_i] = np.sum(output_error[:, kernel_i])

        return dy_dkernel, dy_dbias, dy_dx

In [11]:
# тоже самое, только теперь ручная свертка умеет работать с батчами, np.newaxis теперь не нужен, размер входа будет [BATCH_SIZE, SENTENCE_LEN, EMB_DIM]

sample = to_matrix(np.random.choice(lines, size=5), token_to_id, max_len=130)

emb = Embedding(len(token_to_id), 16)
encoded_data = emb(sample) # батч из 5 предложений

conv = Conv1d(16, 4, 5)
torch_conv = torch.nn.Conv1d(16, 4, 5)
torch_conv.weight.data = torch.as_tensor(conv.kernel)
torch_conv.bias.data = torch.as_tensor(conv.bias)

torch_input = torch.tensor(encoded_data, dtype=torch.float64, requires_grad=True)
torch_out = torch_conv(torch_input.permute(0, 2, 1))  # permute потому что torch на вход принимает формат [BATCH_SIZE, EMB_DIM, SENTENCE_LEN]

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.permute(0, 2, 1).detach().numpy(), conv(encoded_data)))

print("Shape выхода свертки:", conv(encoded_data).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(conv(encoded_data).shape))
check_error_torch = torch.tensor(np.transpose(check_error, (0, 2, 1)))

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = conv.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_conv.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_conv.weight.grad.detach().numpy(), kernel_grad))

Vanilla Forward такой же как и torch forward: True
Shape выхода свертки: (5, 126, 4)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


# linear check

In [12]:
class Linear(BaseLayer):
    """
    Linear class permorms ordinary FC layer in neural networks
    Parameters:
    n_input - size of input neurons
    n_output - size of output neurons
    Methods:
    set_optimizer(optimizer) - is used for setting an optimizer for gradient descent
    forward(x) - performs forward pass of the layer
    backward(output_error, learning_rate) - performs backward pass of the layer
    """

    def __init__(self, n_input: int, n_output: int) -> None:
        super().__init__()
        self.input = None
        self.n_input = n_input
        self.n_output = n_output
        self.w = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(n_input, n_output))
        self.b = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(1, n_output))

        self.w_optimizer = None
        self.b_optimizer = None

    def set_optimizer(self, optimizer) -> None:
        self.w_optimizer = copy.copy(optimizer)
        self.b_optimizer = copy.copy(optimizer)

        self.w_optimizer.set_weight(self.w)
        self.b_optimizer.set_weight(self.b)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        return x.dot(self.w) + self.b

    def backward(self, output_error: np.array) -> np.array:
        # assert self.w_optimizer is not None and self.b_optimizer is not None, 'You should set an optimizer'
        w_grad = self.input.T.dot(output_error)
        b_grad = np.ones((1, len(output_error))).dot(output_error)
        input_error = output_error.dot(self.w.T)

        # self.w = self.w_optimizer.step(w_grad)
        # self.b = self.b_optimizer.step(b_grad)
        return w_grad, b_grad, input_error

In [13]:
# проверка того, что линейный слой работает правильно, транспонирование весов происходит потому, что домножение на веса в моем слое справа

sample = np.random.normal(loc=0, scale=100, size=(100, 16))

linear = Linear(16, 25)
torch_linear = torch.nn.Linear(16, 25)
torch_linear.weight.data = torch.as_tensor(linear.w.T)
torch_linear.bias.data = torch.as_tensor(linear.b)

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = torch_linear(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), linear(sample)))

print("Shape выхода линейного слоя:", linear(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(linear(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = linear.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_linear.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_linear.weight.grad.detach().numpy(), kernel_grad.T))

Vanilla Forward такой же как и torch forward: True
Shape выхода линейного слоя: (100, 25)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [14]:
class Linear3d(BaseLayer):
    """
    Linear class permorms ordinary FC layer in neural networks
    Parameters:
    n_input - size of input neurons
    n_output - size of output neurons
    Methods:
    set_optimizer(optimizer) - is used for setting an optimizer for gradient descent
    forward(x) - performs forward pass of the layer
    backward(output_error, learning_rate) - performs backward pass of the layer
    """

    def __init__(self, n_input: int, n_output: int) -> None:
        super().__init__()
        self.input = None
        self.n_input = n_input
        self.n_output = n_output
        self.w = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(n_input, n_output))
        self.b = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(1, n_output))

        self.w_optimizer = None
        self.b_optimizer = None

    def set_optimizer(self, optimizer) -> None:
        self.w_optimizer = copy.copy(optimizer)
        self.b_optimizer = copy.copy(optimizer)

        self.w_optimizer.set_weight(self.w)
        self.b_optimizer.set_weight(self.b)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        return np.matmul(x, self.w) + self.b  # the same as @

    def backward(self, output_error: np.array) -> np.array:
        # assert self.w_optimizer is not None and self.b_optimizer is not None, 'You should set an optimizer'
        # перемножаем последние 2 измерения друг с другом с помощью matmul и суммируем
        w_grad = np.sum(np.transpose(self.input, (0, 2, 1)) @ output_error, axis=0)
        b_grad = np.sum(output_error, axis=(0, 1))
        input_error = output_error @ self.w.T

        # self.w = self.w_optimizer.step(w_grad)
        # self.b = self.b_optimizer.step(b_grad)
        return w_grad, b_grad, input_error

In [15]:
# проверка того, что линейный слой для 3-х измерений работает правильно, транспонирование весов происходит потому, что домножение на веса в моем слое справа

sample = np.random.normal(loc=0, scale=100, size=(100, 32, 16))

linear = Linear3d(16, 25)
torch_linear = torch.nn.Linear(16, 25)
torch_linear.weight.data = torch.as_tensor(linear.w.T)
torch_linear.bias.data = torch.as_tensor(linear.b)

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = torch_linear(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), linear(sample)))

print("Shape выхода линейного слоя:", linear(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(linear(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = linear.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_linear.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_linear.weight.grad.detach().numpy(), kernel_grad.T))

Vanilla Forward такой же как и torch forward: True
Shape выхода линейного слоя: (100, 32, 25)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [16]:
np.transpose(sample, (0, 2, 1))[1].dot(check_error[1])

array([[-6.01432425e+04,  8.54376029e+04, -2.03543253e+04,
        -4.57779690e+04,  5.93759573e+04,  2.26119128e+04,
        -8.51991229e+04,  5.65570757e+04,  4.50999740e+04,
        -2.50116531e+04,  8.52174259e+04,  2.23094183e+04,
        -8.94199095e+04, -8.94775158e+04,  2.22095782e+04,
        -6.91201388e+04, -4.39176286e+04,  8.89706282e+04,
         5.34499053e+03, -1.94521761e+04,  6.32939855e+04,
         6.96434504e+04,  8.53401340e+04,  5.01143782e+04,
        -7.60784954e+04],
       [ 6.17063591e+04, -4.19896315e+04,  9.91427993e+04,
         1.14313845e+05,  5.81959143e+02, -6.69621451e+03,
         4.03134117e+04, -5.59623141e+04,  9.22817574e+03,
         1.03086575e+04, -9.23303077e+04,  4.44231216e+04,
         8.54574455e+03, -8.02838840e+04, -6.68541904e+04,
         5.35844574e+04,  5.14282893e+04,  6.01799688e+04,
        -2.77477396e+04,  2.56258701e+04,  7.69994258e+04,
        -4.83805158e+04, -1.61227823e+04, -7.18813760e+04,
         1.91162720e+04],
    

In [17]:
(np.transpose(sample, (0, 2, 1))@check_error)[1]

array([[-6.01432425e+04,  8.54376029e+04, -2.03543253e+04,
        -4.57779690e+04,  5.93759573e+04,  2.26119128e+04,
        -8.51991229e+04,  5.65570757e+04,  4.50999740e+04,
        -2.50116531e+04,  8.52174259e+04,  2.23094183e+04,
        -8.94199095e+04, -8.94775158e+04,  2.22095782e+04,
        -6.91201388e+04, -4.39176286e+04,  8.89706282e+04,
         5.34499053e+03, -1.94521761e+04,  6.32939855e+04,
         6.96434504e+04,  8.53401340e+04,  5.01143782e+04,
        -7.60784954e+04],
       [ 6.17063591e+04, -4.19896315e+04,  9.91427993e+04,
         1.14313845e+05,  5.81959143e+02, -6.69621451e+03,
         4.03134117e+04, -5.59623141e+04,  9.22817574e+03,
         1.03086575e+04, -9.23303077e+04,  4.44231216e+04,
         8.54574455e+03, -8.02838840e+04, -6.68541904e+04,
         5.35844574e+04,  5.14282893e+04,  6.01799688e+04,
        -2.77477396e+04,  2.56258701e+04,  7.69994258e+04,
        -4.83805158e+04, -1.61227823e+04, -7.18813760e+04,
         1.91162720e+04],
    

# Softmax (dim=-1) check

In [18]:
class SoftMaxLayer3D(BaseLayer):
    def __init__(self):
        self.input = None
        self.forward_result = None

    def forward(self, x, grad=True):
        self.input = x
        exp = np.exp(x)
        self.forward_result = exp / np.sum(exp, axis=-1, keepdims=True)
        return self.forward_result

    def backward(self, output_error):
        "https://binpord.github.io/2021/09/26/softmax_backprop.html"
        return (output_error - (output_error*self.forward_result).sum(axis=-1, keepdims=True)) * self.forward_result

In [23]:
# проверка того, softmax работает также как и в torch

sample = np.random.normal(loc=0, scale=100, size=(100, 32, 16))

softmax = SoftMaxLayer3D()

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = nn.functional.softmax(torch_input, dim=-1)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), softmax(sample)))

print("Shape выхода слоя:", softmax(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(softmax(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
in_error = softmax.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))

Vanilla Forward такой же как и torch forward: True
Shape выхода линейного слоя: (100, 32, 16)
Градиент по входу совпадает: True


# attention check

In [24]:
class MultiHeadAttentionLayer(BaseLayer):
    def __init__(self, hid_dim: int, n_heads: int) -> None:
        
        assert hid_dim % n_heads == 0
        
        self.input = None
        self.attn_bias = None
        self.attentions = []
        self.q = None
        self.k = None
        self.v = None
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_size = hid_dim // n_heads
        
        self.c_attn = Linear3d(hid_dim, hid_dim * 3)
        self.c_proj = Linear3d(hid_dim, hid_dim)
        self.softmax = SoftMaxLayer3D()

        self.scale = np.sqrt(self.head_size)

    def set_optimizer(self, optimizer) -> None:
        self.c_attn.set_optimizer(optimizer)
        self.c_proj.set_optimizer(optimizer)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        
        q_k_v = self.c_attn(x)
        self.q = q_k_v[:, :, :self.hid_dim]
        self.k = q_k_v[:, :, self.hid_dim:self.hid_dim*2]
        self.v = q_k_v[:, :, self.hid_dim*2:self.hid_dim*3]
        assert self.q.shape == self.k.shape == self.v.shape == x.shape, "q, k and v must have the same shape as x"

        head_outputs = []
        for head_index in range(self.n_heads):
            head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

            head_queries = self.q[..., head_selector]
            head_keys = self.k[..., head_selector]
            head_values = self.v[..., head_selector]

            single_head_output = self._attention_for_head(
                head_queries, head_keys, head_values,
                is_causal=True)
            head_outputs.append(single_head_output)

        combined_head_outputs = np.concatenate(head_outputs, axis=-1)
        return self.c_proj(combined_head_outputs)

    def _attention_for_head(self, query, key, value, is_causal=False):
        L, S = query.shape[-2], key.shape[-2]
        self.attn_bias = np.zeros((L, S), dtype=query.dtype)
        if is_causal:
            temp_mask = np.tril(np.ones((L, S)))
            self.attn_bias = np.where(temp_mask==0, float('-inf'), 0)
            self.attn_bias = self.attn_bias.astype(query.dtype)
    
        attn_weight = query @ key.transpose(0, 2, 1) / self.scale
        attn_weight += self.attn_bias
        
        attn_weight = self.softmax(attn_weight)
        self.attentions.append(attn_weight)
        
        result = attn_weight @ value

        # result = self.softmax((query @ key.transpose(0, 2, 1) / self.scale) + self.attn_bias) @ value
        
        return result

    def _attention_for_head_grad(self, query, key, value, output_error, head_n):
        # looking at the result expression in the forward pass
        cur_attention = self.attentions[head_n]
        
        v_grad = cur_attention.transpose((0, 2, 1)) @ output_error
        input_error = output_error @ value.transpose((0, 2, 1))

        # we use layer, which uses it's states to calculate grad, so we need to set state that was used during forward pass in current head
        self.softmax.forward_result = cur_attention
        softmax_grad = self.softmax.backward(input_error)

        k_grad = softmax_grad.transpose(0, 2, 1) @ query / self.scale # we need to transpose output error due to the fact key was transposed in forward
        q_grad = softmax_grad @ key / self.scale

        return q_grad, k_grad, v_grad

    def backward(self, output_error: np.array) -> np.array:
        c_proj_w_grad, c_proj_b_grad, projection_error = self.c_proj.backward(output_error)
        
        q_grads, k_grads, v_grads = [], [], []
        for head_index in range(self.n_heads):
            # for each head we choose it's error
            head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

            attention_error = projection_error[..., head_selector]
            head_queries = self.q[..., head_selector]
            head_keys = self.k[..., head_selector]
            head_values = self.v[..., head_selector]

            q_grad, k_grad, v_grad = self._attention_for_head_grad(head_queries, head_keys, head_values, attention_error, head_index)
            q_grads.append(q_grad)
            k_grads.append(k_grad)
            v_grads.append(v_grad)

        q_grads = np.concatenate(q_grads, axis=-1)
        k_grads = np.concatenate(k_grads, axis=-1)
        v_grads = np.concatenate(v_grads, axis=-1)

        q_k_v_output = np.concatenate([q_grads, k_grads, v_grads], axis=-1)
        
        c_attn_w_grad, c_attn_b_grad, input_error = self.c_attn.backward(q_k_v_output)
        return input_error

In [44]:
# torch attention
class MaskedSelfAttention(nn.Module):
    def __init__(self, dim: int, num_heads: int):
        super().__init__()
        self.c_attn = nn.Linear(dim, dim * 3)  # query + key + value, combined
        self.c_proj = nn.Linear(dim, dim)  # output projection
        self.dim, self.num_heads = dim, num_heads
        self.head_size = dim // num_heads

        self.scale = torch.sqrt(torch.FloatTensor([self.head_size]))

    # you can choose less effective method and uncomment the code below, it works the same
    # ------------- start of less effective -------------

    # def forward(self, x):
    #     q, k, v = self.c_attn(x).split(dim=-1, split_size=self.dim)
    #     assert q.shape == k.shape == v.shape == x.shape, "q, k and v must have the same shape as x"


    #     # Note: this is an inefficient implementation that uses a for-loop.
    #     # To get the full grade during homework, please re-implement this code:
    #     # 1) do not use for-loops (or other loops). Compute everything in parallel with vectorized operations
    #     # 2) do not use F.scaled_dot_product_attention - write your own attention code using basic PyTorch ops
    #     head_outputs = []
    #     for head_index in range(self.num_heads):
    #         head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

    #         head_queries = q[..., head_selector]
    #         head_keys = k[..., head_selector]
    #         head_values = v[..., head_selector]

    #         single_head_output = self.scaled_dot_product_attention(
    #             head_queries, head_keys, head_values,
    #             is_causal=True)
    #         # docs: https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
    #         head_outputs.append(single_head_output)

    #     combined_head_outputs = torch.cat(head_outputs, dim=-1)
    #     return self.c_proj(combined_head_outputs)

    # def scaled_dot_product_attention(self, query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False, scale=None) -> torch.Tensor:
    #     # Efficient implementation equivalent to the following:
    #     L, S = query.size(-2), key.size(-2)
    #     scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    #     attn_bias = torch.zeros(L, S, dtype=query.dtype)
    #     if is_causal:
    #         assert attn_mask is None
    #         temp_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0)
    #         attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
    #         attn_bias.to(query.dtype)
    
    #     if attn_mask is not None:
    #         if attn_mask.dtype == torch.bool:
    #             attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
    #         else:
    #             attn_bias += attn_mask
    #     attn_weight = query @ key.transpose(-2, -1) * scale_factor
    #     attn_weight += attn_bias
    #     attn_weight = torch.softmax(attn_weight, dim=-1)
    #     # attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    #     return attn_weight @ value

    # ------------- end of less effective -------------

    def forward(self, x):
        # kinda more effective
        batch_size, seq_len, _ = x.shape
        q, k, v = self.c_attn(x).split(dim=-1, split_size=self.dim)
        # [BATCH_SIZE, SEQ_LEN, DIM]
        assert q.shape == k.shape == v.shape == x.shape, "q, k and v must have the same shape as x"
        
        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_SIZE]
        q = q.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, SEQ_LEN]
        energy = torch.matmul(q, k.permute(0, 1, 3, 2)) / self.scale

        # чтобы не смотреть в будущее
        mask = torch.tril(torch.ones((seq_len, seq_len)))
        energy = energy.masked_fill(mask == 0, float('-inf'))

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, SEQ_LEN]
        probs = nn.functional.softmax(energy, dim=-1)

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_SIZE]
        output = torch.matmul(probs, v)

        # [BATCH_SIZE, SEQ_LEN, NUM_HEADS, HEAD_SIZE]
        output = output.permute(0, 2, 1, 3).contiguous()

        # [BATCH_SIZE, SEQ_LEN, DIM]
        output = output.view(batch_size, -1, self.dim)

        # [BATCH_SIZE, SEQ_LEN, DIM]
        output = self.c_proj(output)
        return output

In [45]:
# проверка того, что multihead masked attention работает также как и в torch

for head_n in [1, 3, 6]:
    print('Число голов:', head_n)
    sample = np.random.randn(100, 32, 36)
    
    attention = MultiHeadAttentionLayer(36, head_n)
    torch_attention = MaskedSelfAttention(36, head_n)
    # ставим одинаковые иходные веса для слоев
    attention.c_attn.w = torch_attention.c_attn.weight.data.numpy().T
    attention.c_attn.b = torch_attention.c_attn.bias.data.numpy().reshape(1, -1)
    attention.c_proj.w = torch_attention.c_proj.weight.data.numpy().T
    attention.c_proj.b = torch_attention.c_proj.bias.data.numpy().reshape(1, -1)

    attention_out = attention(sample)
    torch_input = torch.tensor(sample, dtype=torch.float32, requires_grad=True)
    torch_out = torch_attention(torch_input)
    
    # проверка что forward работает также как у torch, 
    print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), attention_out, atol=1e-6))
    
    print("Shape выхода слоя:", attention_out.shape)
    
    # случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
    check_error = np.random.randn(100, 32, 36)
    check_error_torch = torch.tensor(check_error)
    
    torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
    in_error = attention.backward(check_error)  # тоже самое, только в ручном слое
    
    # проверка градиентов весов и входа
    print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error, atol=1e-6))
    print('='*30)

Число голов: 1
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True
Число голов: 3
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True
Число голов: 6
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True
